# Python - Lập trình hướng đối tượng

## Class variables (Class attributes)

Class variables được khai báo bên ngoài tất cả phương thức, là dữ liệu thuộc về lớp, được chia sẻ bởi tất cả đối tượng sinh ra từ lớp đó

In [1]:
class Person:
    species = "Homo Sapiens"
    count = 0

    def __init__(self, name, age):
        self.name = name
        self.age = age
        Person.count += 1

ba = Person("Ba", 29)
beo = Person("Béo", 29)

print(Person.species)
print(ba.species)
print(beo.species)

print(Person.count)

print(id(ba.species))
print(id(beo.species))

Homo Sapiens
Homo Sapiens
Homo Sapiens
2
4386543664
4386543664


In [9]:
class BankAccount:
    rate = 0.05
    minimum_balance = 50000

    def __init__(self, account_number, account_name, balance=0):
        self._account_number = account_number
        self._account_name = account_name
        self._balance = balance

    def display(self):
        print(self._account_number, self._account_name, self._balance, "₫")

    def withdraw(self, amount):
        self._balance -= amount

    def deposite(self, amount):
        self._balance += amount


my_account = BankAccount(1, "Ba", 0)

## Class methods

Tương tự class variables, class methods là các phương thức thuộc về lớp, tham số đầu tiên theo quy ước là `cls` tham chiếu đến lớp thay vì instance

In [18]:
class Person:
    species = "Homo Sapiens"
    count = 0

    def __init__(self, name):
        self.name = name
        Person.count += 1

    @classmethod
    def show_count(cls):
        print(f"There are {cls.count} {cls.species} on earth")


Person.show_count()

adam = Person("Adam")
eva = Person("Eva")

Person.show_count()

There are 0 Homo Sapiens on earth
There are 2 Homo Sapiens on earth


Class methods hữu ích trong việc tạo Factory methods

💡 Factory methods là một design pattern để tạo ra các đối tượng (thường được gọi là sản phẩm - product) theo các cách thức cụ thể

In [2]:
s = "Ba, 29"
d = {"name": "Béo", "age": 92}

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def from_str(cls, s):
        name, age = s.split(",")
        return cls(name, int(age))

    @classmethod
    def from_dict(cls, d):
        return cls(**d)
    
    def display(self):
        print(self.name, self.age)

ba = Person.from_str(s)
ba.display()

beo = Person.from_dict(d)
beo.display()

Ba 29
Béo 92


### Exercise

Thêm 2 phương thức cho class `BankAccount` để khởi tạo từ file csv hoặc json, đầu vào là tên file, kết quả trả về một list.

Chỉnh sửa phương thức `display()` để in ra thông tin theo dạng bảng

In [20]:
import os
import csv
class BankAccount:
    def __init__(self, account_number, account_name, balance=0):
        self.account_number = account_number
        self.account_name = account_name
        self.balance = balance

    @classmethod
    def from_csv(cls, csv_file):
        accounts = []

        with open(csv_file) as file:
            reader = csv.reader(file)

            for account_number, account_name, balance in reader:
                accounts.append(
                    cls(account_number, account_name, int(balance)))

        return accounts

    def display(self):
        for account in self:
            account.display()
            print(account.account_number, account.account_name, account.balance)

file = os.path.abspath("bank_accounts.csv")     
bank_accounts = BankAccount.from_csv(file)
bank_accounts.display()

AttributeError: 'list' object has no attribute 'display'

Path of the file.. c:\Users\thuy.nguyen6_onemoun\PYTHON_OMD\start_python\python_oop\bank_accounts.csv


## Magic methods

Magic cung cấp một cách thức đơn giản hơn để làm việc với các objects

Để tùy chỉnh giá trị trả về khi chuyển đổi object thành chuỗi, sử dụng magic method `__repr__()`

In [10]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __gt__(self, other):
        return self.x > other.x and self.y > other.y

    def __ge__(self, other):
        return self.x >= other.x and self.y >= other.y

    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

p1 = Point(0, 0)
p2 = Point(1, 1)

print(p1)
print(p1 < p2)
print(p1 <= p2)
print(p1 + p2)

Point(0, 0)
True
True
Point(1, 1)


In [26]:
class TagCloud:
    def __init__(self):
        self.tags = {}

    def add(self, tag):
        self.tags[tag.lower()] = self.tags.get(tag.lower(), 0) + 1

    def __getitem__(self, tag):
        return self.tags.get(tag.lower(), 0)

    def __setitem__(self, tag, value):
        self.tags[tag.lower()] = self[tag.lower()] + value

    def __iter__(self):
        return iter(self.tags)

tags = TagCloud()
tags.add("Python")
tags.add("python")
tags["magic methods"] = 10
print(tags["python"])
print(tags["magic methods"])
print(tags["none key"])

for tag in tags:
    print(tag, tags[tag])

2
10
0
python 2
magic methods 10


### Exercise

Tạo class `Fraction` (phân số)

- Hàm khởi tạo nhận 2 giá trị `nr` (tử số) và `dr` (mẫu số)
- Nếu `dr` âm, chuyển dấu cho `nr` (VD: 1/-2 => -1/2)
- Triển khai phương thức phù hợp để in ra phân số (VD: `print(fr)` => `-1/2`)
- Viết hàm `hcf` tìm ước chung lớn nhất của `nr` và `dr`
- Thêm phương thức `reduce` rút gọn phân số (gọi trong `__init__`)
- Nếu `nr == 0`, chỉ in ra `0`
- Nếu `dr == 0`, raise ZeroDevisonError
- Nếu `dr == 1`, chỉ in ra `nr`
- Triển khai các phương thức phù hợp cho phép +-*/ với 2 `Fraction` hoặc 1 `Fraction` với 1 số (`int` hoặc `float`), kết quả trả về 1 `Fraction` mới

In [23]:
from solutions.fraction import Fraction

fr = Fraction(1, 2)
other = Fraction(1.5, -3)
print(fr, other)

print()

print(fr + other)
print(fr - other)
print(fr * other)
print(fr / other)

print()

fr = Fraction(1, 2)
print(fr + 1)
print(fr - 1.5)
print(fr * 2)
print(fr / 2)

1/2 -1/2

0
1
-1/4
-1

3/2
-1
1
1/4
